## Modelagem de absorvedores de membrana

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from macroscopic_models import delany_bazley, jca

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
# define frequency space
f_start = 20
f_end = 10000
f = np.arange(f_start, f_end, step=1)
omega = 2 * np.pi * f

# air properties
rho0 = 1.21  # [kg/m³] air density
c0 = 343  # [m/s] sound speed
k0 = omega / c0

### 1. Variação da massa

### 1.1. Influência da espessura da membrana

In [ ]:
# membrane properties
D = 0.10  # [m] size of the air gap
membrane_thicknesses = np.array(
    [0.1e-3, 0.3e-3, 0.5e-3, 0.9e-3]
)  # [m] membrane thickness
rho_m = 1300  # [kg/m^3] specific mass of the membrane
surface_densities = membrane_thicknesses * rho_m  # [kg/m^2]

# porous material properties
d = 0.07  # [m] depth of porous material
sigma = 15000  # [N*s/m^4] flux resistivity
Zp, kp = jca(
    f, sigma, tortuosity=1.1, porosity=0.96, viscous_characteristic_length=1e-4
)  # JCA parameters

# plot setup
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_title(r"Efeito da variação da espessura da membrana no coeficiente de absorção")
ax.set_xlabel("Frequência [Hz]")
ax.set_ylabel(r"$\alpha$ [-]")
ax.set_xlim(f_start, f_end)
ax.set_ylim(-0.2, 1.2)
ax.grid(axis="both", which="both", linestyle="--", alpha=0.4)

labels = [
    r"$t = 0.1~[\text{mm}]$",
    r"$t = 0.3~[\text{mm}]$",
    r"$t = 0.5~[\text{mm}]$",
    r"$t = 0.9~[\text{mm}]$",
]


for surface_density, label in zip(surface_densities, labels):
    # estimating the ressonance frequency —— in reality, it will be somewhere between f50 and f60
    f50 = 50 / np.sqrt(surface_density * D)
    f60 = 60 / np.sqrt(surface_density * D)
    Zm = 1j * omega * surface_density  # membrane impedance

    # fmt: off
    Zsp = -1j*Zp / np.tan(kp*d) # impedance on top of the porous material layer
    Zsi = (((-1j*Zsp*rho0*c0) / (np.tan(k0*(D - d)))) + (rho0*c0)**2) / (Zsp - ((1j*rho0*c0) / (np.tan(k0*(D - d))))) # impedance on top of the air layer
    # fmt: on

    Zs = Zm + Zsi  # total impedance (series association)

    Vp = (Zs - (rho0 * c0)) / (Zs + (rho0 * c0))  # reflection coefficient
    alpha = 1 - (abs(Vp)) ** 2  # absorption coefficient
    ax.semilogx(f, alpha, label=label)
ax.legend()
plt.show()